In [2]:
import json
import requests
import threading
import re
import traceback

URL = 'http://3.84.244.86:8080/enhancer/chain/PurchaseOrder'
FLATTEN_URL = 'http://34.80.26.185:8086/PO_Processing/GetDataSolr.flatteringTemp'
CHOOSE_URL = 'http://34.80.26.185:8086/PO_Processing/GetDataSolr.Temp'
VALIDATION_URL = 'http://34.80.26.185:8086/PO_Processing/ReArrangingofData'
REGEX_URL = 'http://35.221.160.146:5020/regexResult'
OVERLAP_URL = 'http://35.186.166.22:8082/portal/servlet/service/Poheader.poi'

LABEL = 'http://fise.iks-project.eu/ontology/entity-reference'
TEXT_LABEL = 'http://fise.iks-project.eu/ontology/selected-text'
RELATION_LABEL = 'http://purl.org/dc/terms/relation'
SELECTED_LABEL = 'http://bizlem.io/PurchaseOrderProcessing#'

In [3]:
def middle(value, left, right):
    return value >= left and value <= right

def get_x(obj, index=0):
    return obj['bounding_poly']['vertices'][index]['x']

def get_y(obj, index=0):
    return obj['bounding_poly']['vertices'][index]['y']

def to_float(s):
    s_format = ''.join(re.split('[^\d\.]', s))
    return float(s_format)

In [14]:
folder = 'c55'
path = 'p4materials/' + folder + '.json'
# path = '/home/pexea12/N2000HZ118-19-0149628.json'
with open(path) as f:
    data = json.load(f)
    data_sum = data[0]
    data = data[1:]

In [16]:
def parseJSON(data, x_thres, y_thres):
    response = []
    
    # Build histogram
    y = [ (get_y(obj) + get_y(obj, 2)) // 2 for obj in data ]
    y.sort()

    cur_y = y[0]
    hist_y = { cur_y: cur_y }

    for yc in y:
        if yc == cur_y: 
            continue
        if yc - cur_y > y_thres:
            cur_y = yc
        hist_y[yc] = cur_y

    data.sort(key=lambda obj: hist_y[(get_y(obj) + get_y(obj, 2)) // 2] * 1000000 + get_x(obj))
    

    curLineYCoord = (get_y(data[0]) + get_y(data[0], 2)) // 2
    curXEnd = get_x(data[0], 2)
    curXStart = get_x(data[0])
    line = {
        'y': curLineYCoord,
        'words': [],
    }

    completedWord = data[0]['description']
    characters = set(['+', '-', ';', '-', '\\', "'", '"', '/'])
    open_chars = set(['(', '[', '{'])
    close_chars = set([')', ']', '}'])

    for i in range(1, len(data)):
        curWord = data[i]['description']
        yMid = (get_y(data[i]) + get_y(data[i], 2)) // 2
        xStart = get_x(data[i])


        if hist_y[yMid] <= hist_y[curLineYCoord] + y_thres and hist_y[yMid] >= hist_y[curLineYCoord] - y_thres:
            if xStart <= curXEnd + x_thres \
                or (xStart <= curXEnd + 2 * x_thres and ( \
                    curWord[0] in characters \
                    or curWord[0] in close_chars \
                    or ((curWord[0] in set([',', '.', '%'])) \
                        and '0' <= completedWord[-1] and completedWord[-1] <= '9') \
                    or completedWord[-1] in characters \
                    or completedWord[-1] in open_chars \
                    or (len(completedWord) >= 2 and completedWord[-1] in set([',', '.']) \
                        and '0' <= curWord[0] and curWord[0] <= '9' \
                        and '0' <= completedWord[-2] and completedWord[-2] <= '9'))):
                    completedWord += curWord
                    curXEnd = get_x(data[i], 2)
            else:
                line['words'].append({
                    'word': completedWord,
                    'x1': curXStart,
                    'x2': curXEnd,
                })

                curXEnd = get_x(data[i], 2)
                completedWord = data[i]['description']
                curXStart = get_x(data[i])
        else:
            line['words'].append({
                'word': completedWord,
                'x1': curXStart,
                'x2': curXEnd,
            })

            response.append(line)
            curLineYCoord = (get_y(data[i]) + get_y(data[i], 2)) // 2
            curXStart = get_x(data[i])
            curXEnd = get_x(data[i], 2)

            completedWord = data[i]['description']
            line = {
                'y': curLineYCoord,
                'words': [],
            }

    return response

In [17]:
slist = parseJSON(data, 5, 10)

for line_index, line in enumerate(slist):
    s = ' '.join([ word['word'] for word in line['words'] ])
    
    print(line_index, line['y'], s)
    
max_y = slist[-1]['y']
max_x = 0
for line in slist:
    if len(line['words']) == 0:
        continue
        
    last_word = line['words'][-1]['x2']
    if last_word > max_x:
        max_x = last_word
        
print('max', max_x, max_y)

0 97 ORIGINAL FOR RECIPIENT
1 120 GST NO :24AABCE8234A127 TAX INVOICE
2 132 Tax Invoice No: DR2018000084
3 164 ESSAR POWER HAZIRA LIMITED
4 186 PAN : ESSAR
5 188 AABCE8234A 27,KM
6 200 SURATHAZIRA ROAD,HAZIRA-394270 Invoice Date: 01-02-2019
7 210 POWER
8 253 -
9 268 Details ofReceiver (Billed to): Details of Consignee -
10 280 (Shipped to):
11 306 Name: ESSAR STEEL INDIA LIMITED Name: ESSAR STEEL INDIA LIMITED
12 325 -
13 343 Address: 27th KM Surat-Hazira Road, Dist. Surat,Hazira -394270. Address: 27th KM Surat-Hazira Road , Dist. Surat,Hazira-394270.
14 371 -
15 395 .
16 409 State: Gujarat State: Gujarat
17 420 -
18 446 State Code: 24 State Code: 24
19 468 -
20 482 GSTIN /Unique ID: 24AAACE1741P1ZN GSTIN /Unique ID : 24AAACE1741P1ZN .
21 507 E
22 522 Date of Removal ofGoods: 01-02-2019 Customer P. O. No & Date PPA Dt. 10.03.2010 & Ammended PPA Dt. 19.03.2013 &
23 556 Time ofRemoval ofGoods: AmmendedPPA Dt. 30.03.2017 & AmmendedNo. 3 PPA Dt. 15.05.2018
24 591 Vehicle No: NA Mode of Tra

In [18]:
for i, line in enumerate(slist):
    print(i, line['y'], [ word['word'] for word in line['words'] ])

0 97 ['ORIGINAL', 'FOR', 'RECIPIENT']
1 120 ['GST', 'NO', ':24AABCE8234A127', 'TAX', 'INVOICE']
2 132 ['Tax', 'Invoice', 'No:', 'DR2018000084']
3 164 ['ESSAR', 'POWER', 'HAZIRA', 'LIMITED']
4 186 ['PAN', ':', 'ESSAR']
5 188 ['AABCE8234A', '27,KM']
6 200 ['SURATHAZIRA', 'ROAD,HAZIRA-394270', 'Invoice', 'Date:', '01-02-2019']
7 210 ['POWER']
8 253 ['-']
9 268 ['Details', 'ofReceiver', '(Billed', 'to):', 'Details', 'of', 'Consignee', '-']
10 280 ['(Shipped', 'to):']
11 306 ['Name:', 'ESSAR', 'STEEL', 'INDIA', 'LIMITED', 'Name:', 'ESSAR', 'STEEL', 'INDIA', 'LIMITED']
12 325 ['-']
13 343 ['Address:', '27th', 'KM', 'Surat-Hazira', 'Road,', 'Dist.', 'Surat,Hazira', '-394270.', 'Address:', '27th', 'KM', 'Surat-Hazira', 'Road', ',', 'Dist.', 'Surat,Hazira-394270.']
14 371 ['-']
15 395 ['.']
16 409 ['State:', 'Gujarat', 'State:', 'Gujarat']
17 420 ['-']
18 446 ['State', 'Code:', '24', 'State', 'Code:', '24']
19 468 ['-']
20 482 ['GSTIN', '/Unique', 'ID:', '24AAACE1741P1ZN', 'GSTIN', '/Unique', '

In [19]:
sres = []
sres_words = []

for line in slist:
    sres.append({
        'y': line['y'],
        'words': [ word['word'] for word in line['words'] ],
    })
    
    sres_words.append({
        'y': line['y'],
        'words': [ word['word'] for word in line['words'] ]
    })
    

def request_line_and_replace(slist, sres, sres_words, line_index):
    sentence = ' '.join([ word for word in sres[line_index]['words'] ])
    res = {}
    try: 
        r = requests.post(URL, data=sentence.encode('utf-8'), headers={'Content-Type': 'application/pdf'})
        r = r.json()

        for obj in r:
            if LABEL in obj and RELATION_LABEL in obj:
                res[obj[LABEL][0]['@id']] = obj[RELATION_LABEL][0]['@id']

        for obj in r:
            if TEXT_LABEL in obj and '@id' in obj:
                obj_id = obj['@id']
                for kurl, vid in res.items():
                    if vid == obj_id:
                        res[kurl] = obj[TEXT_LABEL][0]['@value']
       
        merging_r = {}
        words = slist[line_index]['words']
        
        word_index = 0
        while word_index < len(words):
            word = words[word_index]
            found = False
            
            for kurl, vtext in res.items():
                if kurl in merging_r:
                    continue
                    
                if found:
                    break
                    
                vtext_first_word = vtext.split(' ')[0]
                
                if vtext_first_word not in word['word']:
                    continue
                
                current_word_indexes = [ word_index ]
                current_s = word['word']
                current_i = word_index + 1
                while len(current_s) < len(vtext) and current_i < len(words):
                    current_word_indexes.append(current_i)
                    current_s += ' ' + words[current_i]['word']
                    current_i += 1
                
                if vtext not in current_s:
                    continue
                
                found = True
                merging_r[kurl] = current_word_indexes
                
            if not found:
                word_index += 1
        
#         print(merging_r)

        removed_words = set()
        for url, merging_arr in merging_r.items():
            min_index = merging_arr[0]
            start_word = slist[line_index]['words'][min_index]
            sres_words[line_index]['words'][min_index] = url
            
            for word_index in merging_arr[1:]:
                removed_words.add(word_index)
                current_word = slist[line_index]['words'][word_index]
                start_word['word'] += ' ' + current_word['word']
                start_word['x2'] = current_word['x2']
        
        slist_newarr = []
        sres_words_newarr = []
        
        for word_index, word in enumerate(slist[line_index]['words']):
            if word_index in removed_words: continue
            
            slist_newarr.append(word)
            sres_words_newarr.append(sres_words[line_index]['words'][word_index])
            
        slist[line_index]['words'] = slist_newarr
        sres_words[line_index]['words'] = sres_words_newarr
        
        sres[line_index]['words'] = list(res.keys())
#         print(sres_words_newarr)
                
    except Exception as e:
        traceback.print_exc()
                    
        
def request_all(sres):
    threads = [ threading.Thread(target=request_line_and_replace, args=(slist, sres, sres_words, line_index)) for line_index in range(len(sres)) ]
    for thread in threads:
        thread.start()
        
    for thread in threads:
        thread.join()

print('requesting...')
request_all(sres)
# request_line_and_replace(slist, sres, sres_words, 1)
print('finish requesting...')

# print(slist[1]['words'])
# print(sres_words[1]['words'])
for line_index, line in enumerate(sres):
    print(line_index, line['y'])
    print('\t', line['words'])
    print('\t', [ word['word'] for word in slist[line_index]['words'] ])
    print('\t', sres_words[line_index]['words'])

requesting...
finish requesting...
0 97
	 []
	 ['ORIGINAL', 'FOR', 'RECIPIENT']
	 ['ORIGINAL', 'FOR', 'RECIPIENT']
1 120
	 ['http://bizlem.io/PurchaseOrderProcessing#Invoice_Number', 'http://bizlem.io/PurchaseOrderProcessing#TaxInvoice']
	 ['GST', 'NO :24AABCE8234A127 TAX INVOICE']
	 ['GST', 'http://bizlem.io/PurchaseOrderProcessing#Invoice_Number']
2 132
	 ['http://bizlem.io/PurchaseOrderProcessing#TaxInvoice']
	 ['Tax Invoice', 'No:', 'DR2018000084']
	 ['http://bizlem.io/PurchaseOrderProcessing#TaxInvoice', 'No:', 'DR2018000084']
3 164
	 []
	 ['ESSAR', 'POWER', 'HAZIRA', 'LIMITED']
	 ['ESSAR', 'POWER', 'HAZIRA', 'LIMITED']
4 186
	 ['http://bizlem.io/PurchaseOrderProcessing#PAN_Number']
	 ['PAN', ':', 'ESSAR']
	 ['http://bizlem.io/PurchaseOrderProcessing#PAN_Number', ':', 'ESSAR']
5 188
	 []
	 ['AABCE8234A', '27,KM']
	 ['AABCE8234A', '27,KM']
6 200
	 ['http://bizlem.io/PurchaseOrderProcessing#Invoice_Date']
	 ['SURATHAZIRA', 'ROAD,HAZIRA-394270', 'Invoice Date:', '01-02-2019']
	 ['SUR

In [20]:
for line_index, line in enumerate(sres_words):
    for word in line['words']:
        if SELECTED_LABEL in word:
            print(line_index, line['y'], word)

1 120 http://bizlem.io/PurchaseOrderProcessing#Invoice_Number
2 132 http://bizlem.io/PurchaseOrderProcessing#TaxInvoice
4 186 http://bizlem.io/PurchaseOrderProcessing#PAN_Number
6 200 http://bizlem.io/PurchaseOrderProcessing#Invoice_Date
20 482 http://bizlem.io/PurchaseOrderProcessing#GSTIN
22 522 http://bizlem.io/PurchaseOrderProcessing#Date
22 522 http://bizlem.io/PurchaseOrderProcessing#Purchase_Order_Number
24 591 http://bizlem.io/PurchaseOrderProcessing#Transport
25 630 http://bizlem.io/PurchaseOrderProcessing#HSN/SAC
25 630 http://bizlem.io/PurchaseOrderProcessing#Rate
25 630 http://bizlem.io/PurchaseOrderProcessing#CGST
25 630 http://bizlem.io/PurchaseOrderProcessing#SGST
25 630 http://bizlem.io/PurchaseOrderProcessing#IGST
25 630 http://bizlem.io/PurchaseOrderProcessing#GrossTotal
26 642 http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices
26 642 http://bizlem.io/PurchaseOrderProcessing#Quantity
26 642 http://bizlem.io/PurchaseOrderProcessing#Unit
26 642 http://bizlem

In [21]:
def concat(slist, line_index, index, n_lines=2):
    paragraph = ' '.join([ word['word'] for word in slist[line_index]['words'][index:] ]) + '\n'
    current_word = slist[line_index]['words'][index]
    
    for line in slist[line_index + 1:(line_index + 1) + n_lines]:
        for word in line['words']:
            if word['x1'] >= current_word['x1'] - 15:
                paragraph += word['word'] + ' '
        paragraph += '\n'
    
    return paragraph

def strip(s):
    return ''.join(re.split('[^a-zA-Z0-9]', s.lower()))


date_regex = '\d{1,2}\s?[\.\-\/\:]\s?\d{1,2}\s?[\.\-\/\:]\s?\d{4}|\d{1,2}[\s\-]+[A-Za-z]{3,}[\s\-]+\d{4}|\d{1,2}\s?\/\s?\d{1,2}\s?\/\s?\d{2}'
# (MM.DD.YYYY) | MM - DDD - YYYY | MM - DD - YYYY | MM / DD / YY
number_regex = '(?<![\.\d])(?:\,? ?\d+ ?)+(?:\. ?\d+)?(?!\d{0,} ?%)'
percentage_regex = '\d+\s?%|\d+\s?\.\s?%'

# processing text
patterns = {                                               
    'PAN_Number': [{                                               
        'regex': '[A-Z]{5}[0-9]{4}[A-Z]{1}',
        'main_key': 'PAN_Number',
    }],
    'TAN': [{                                               
        'regex': '[A-Z]{4}[0-9]{5}[A-Z]{1}',
        'main_key': 'TAN',
    }],
    'Purchase_Order_Number': [{                                   
        'regex': '\d{1,}',
        'main_key': 'Purchase_Order_Number',
    }],
    'GSTIN': [{
        'regex': '[A-Z0-9]{15}', 
        'main_key': 'GSTIN',
    }],
    'Invoice_Number': [{
        'regex': '\d{2,}|[A-Z]+\d+[A-Z]\s?(\d{2})?\s?\-\s?\d+|[A-Z]\s?\-?\s?\d+|[A-Z]+\/\d+\-\d+\/\d+(\/\d+)?',
        'main_key': 'Invoice_Number',
    }],
    # 12345 | SA18Y 09 - 00205 | S - 168 | GST/18-19/025
    'Invoice_Date': [{
        'regex': date_regex,
        'main_key': 'Invoice_Date',
    }],
    'Date': [{
        'regex': date_regex,
        'main_key': 'Invoice_Date',
    }],
    'L.R._No': [{
        'regex': '\d{1,2}\s?[(th)\-\.]?\s?([A-Za-z]{3,})\s?[\-\.]?\s?\d{4}|\d{1,2}\s?\.\s?\d{1,2}\s?\.\s?\d{2}|BY MONTH\s?\/?\s?\d{4}\s?\-\s?\d{4}',
        'main_key': 'L.R._No',
    }],
    'L.R._Dt': {
        'regex': date_regex,
        'main_key': 'L.R._Dt',
    },
    'GrossTotal': [{
        'regex': number_regex,
        'main_key': 'GrossTotal',
    }],
}


patterns_reversed = {
    'SGST': [
        {
            'regex': number_regex,
            'main_key': 'SGST_Amount',
        },
        {
            'regex': percentage_regex,
            'main_key': 'SGSTPercentage',
        },
    ],
    'CGST': [
        {
            'regex': number_regex,
            'main_key': 'CGST_Amount',
        },
        {
            'regex': percentage_regex,
            'main_key': 'CSGTPercentage',
        },
    ],
    'IGST': [
        {
            'regex': number_regex,
            'main_key': 'IGST_Amount',
        },
        {
            'regex': percentage_regex,
            'main_key': 'IGSTPercentage',
        },
    ],
    'SGSTPercentage': [{
        'regex': percentage_regex,
        'main_key': 'SGSTPercentage'
    }],
    'CGSTPercentage': [{
        'regex': percentage_regex,
        'main_key': 'CGSTPercentage'
    }],
    'IGSTPercentage': [{
        'regex': percentage_regex,
        'main_key': 'IGSTPercentage'
    }],
}

In [11]:
result_keys = [
    'TaxInvoice',
    'Invoice_Number',
    'Ship_To',
    'Bill_To',
    'EssarGSTIN',
    'Hazira',
    'Invoice_Date',
    'PAN_Number',
    'TAN',
    'Purchase_Order_Number',
    'GSTIN',
    'L.R._No',
    'L.R._Dt',
    'Transport',
    'SERVICES_TAX_REG_NO',
    'SGST_Amount',
    'CGST_Amount',
    'IGST_Amount',
    'CGSTPercentage',
    'SGSTPercentage',
    'IGSTPercentage',
    'GrossTotal',
]

number_keys = [
    'SGST_Amount',
    'CGST_Amount',
    'IGST_Amount',
]

result = { }
gross_total = 0
for key in result_keys:
    result[SELECTED_LABEL + key] = None

for line_index, line in enumerate(slist):
    if len(sres[line_index]['words']) == 0:
        continue

    for word_index, word in enumerate(slist[line_index]['words']):
        word_url = sres_words[line_index]['words'][word_index]
        if SELECTED_LABEL not in word_url:
            continue
        
        word_url_key = word_url.split('#')[-1]
        
        
        if word_url_key == 'TaxInvoice':
            result[word_url] = True
            continue
            
        if word_url_key not in patterns: continue
            
        paragraph = concat(slist, line_index, word_index)
        pattern_list = patterns[word_url_key]
        
        for pattern in pattern_list:
            result_labels = set([ label.split('#')[-1] for label in result.keys() if result[label] is not None ])
            
            # ignore already exist key in result
            if pattern['main_key'] in result_labels:
                continue

            print(line_index, line['y'], pattern['main_key'])

            if pattern['main_key'] == 'GrossTotal':
                for sub_line in slist[line_index:line_index + 3]:
                    start_index = 0
                    for index, word in enumerate(sub_line['words']):
                        if word['x1'] > max_x * 0.7:
                            start_index = index
                            break

                    sentence = ' '.join([ word['word'] for word in sub_line['words'][start_index:] ])
                    search_result = [ to_float(num) for num in re.findall(pattern['regex'], sentence) ]
                    if len(search_result) > 0 and max(search_result) > gross_total:
                        gross_total = max(search_result)

            else:
                search_p = re.search(pattern['regex'], paragraph)
                if search_p is not None:
                    result[SELECTED_LABEL + pattern['main_key']] = search_p.group(0)
        
                
result[SELECTED_LABEL + 'GrossTotal'] = gross_total
                    

for line_index in range(len(slist) - 1, -1, -1):
    if len(sres[line_index]['words']) == 0:
        continue
        
    for word_index, word in enumerate(slist[line_index]['words']):
        word_url = sres_words[line_index]['words'][word_index]
        if SELECTED_LABEL not in word_url:
            continue
        
        word_url_key = word_url.split('#')[-1]
        if word_url_key not in patterns_reversed: continue
            
        paragraph = concat(slist, line_index, 0)
        pattern_list = patterns_reversed[word_url_key]
        
        for pattern in pattern_list:
            result_labels = set([ label.split('#')[-1] for label in result.keys() if result[label] is not None ])
            
            # ignore already exist key in result
            if pattern['main_key'] in result_labels:
                continue

            print(line_index, line['y'], pattern['main_key'])

            search_p = re.search(pattern['regex'], paragraph)
            if search_p is not None:
                result[SELECTED_LABEL + pattern['main_key']] = search_p.group(0)
                

for key in number_keys:
    if result[SELECTED_LABEL + key] is not None:
        result[SELECTED_LABEL + key] = to_float(result[SELECTED_LABEL + key])

print(json.dumps(result, indent=2))

4 360 GSTIN
8 450 Purchase_Order_Number
10 556 Invoice_Date
18 982 GSTIN
22 1152 GrossTotal
47 3282 SGST_Amount
47 3282 SGSTPercentage
45 3282 CGST_Amount
45 3282 CSGTPercentage
20 3282 CSGTPercentage
20 3282 SGSTPercentage
{
  "http://bizlem.io/PurchaseOrderProcessing#TaxInvoice": true,
  "http://bizlem.io/PurchaseOrderProcessing#Invoice_Number": null,
  "http://bizlem.io/PurchaseOrderProcessing#Ship_To": null,
  "http://bizlem.io/PurchaseOrderProcessing#Bill_To": null,
  "http://bizlem.io/PurchaseOrderProcessing#EssarGSTIN": null,
  "http://bizlem.io/PurchaseOrderProcessing#Hazira": null,
  "http://bizlem.io/PurchaseOrderProcessing#Invoice_Date": "22/09/2018",
  "http://bizlem.io/PurchaseOrderProcessing#PAN_Number": null,
  "http://bizlem.io/PurchaseOrderProcessing#TAN": null,
  "http://bizlem.io/PurchaseOrderProcessing#Purchase_Order_Number": "4500251154",
  "http://bizlem.io/PurchaseOrderProcessing#GSTIN": "24AAACE1741P1ZN",
  "http://bizlem.io/PurchaseOrderProcessing#L.R._No": nul

In [20]:
header_keys = set([
    'SNo',
    'DescriptionOfServices',
    'HSN/SAC',
    'Unit',
    'Quantity',
    'Rate',
    'AmountNumbers',
    'SGST',
    'CGST',
    'IGST',
])


selected = []
header_map_arr = []


end = False

for line_index, words in enumerate(slist):
    if len(sres[line_index]['words']) == 0: continue
        
    for word_index, word in enumerate(words['words']):
        url = sres_words[line_index]['words'][word_index]
        if SELECTED_LABEL not in url: continue
            
        url_key = url.split('#')[-1]
        if url_key not in header_keys: continue
            
        x1, x2 = word['x1'], word['x2']
        
        header_map_arr.append((
            SELECTED_LABEL + url_key,
            line_index,
            words['y'],
            x1,
            x2,
        ))

freq = [0] * max_y

for (url_key, _, y, _, _) in header_map_arr:
    start_y = max(0, y - 30)
    stop_y = min(max_y, y + 30)
    for i in range(start_y, stop_y + 1):
        freq[i] += 1

clusters_top = []
for i in range(1, max_y - 1):
    if freq[i] > freq[i - 1] and freq[i] > freq[i + 1]:
        clusters_top.append((i, freq[i]))
    elif freq[i] > freq[i - 1] and freq[i] == freq[i + 1]:
        for j in range(i + 1, max_y - 1):
            if freq[j] < freq[i]:
                clusters_top.append((i, freq[i]))
                break
            elif freq[j] > freq[i]:
                break

clusters_top.sort(key=lambda x: -x[1])
print(clusters_top)

cluster_index = 0
description_index = None
amount_index = None
cluster_headers = []
max_cluster_headers = []
amount_cluster_headers = []

while cluster_index < len(clusters_top): 
    current_y = clusters_top[cluster_index][0]
    for (url_key, line_index, y, x1, x2) in header_map_arr:
        if y >= current_y - 30 and y <= current_y + 30:
            cluster_headers.append((url_key, line_index, y, x1, x2))
            if 'Description' in url_key:
                description_index = cluster_index
            if 'Amount' in url_key:
                amount_index = cluster_index

    if cluster_index == 0:
        max_cluster_headers = cluster_headers
        
    if amount_index is not None:
        amount_cluster_headers = cluster_headers

    if description_index is not None:
        break
    else:
        cluster_index += 1
        cluster_headers = []
        
if description_index is None:
    if amount_index is not None:
        cluster_headers = amount_cluster_headers
    else:
        cluster_headers = max_cluster_headers
        
main_table_headers = cluster_headers

print(json.dumps(main_table_headers, indent=2))

[(1122, 6), (1176, 4), (2514, 2), (2867, 2)]
[
  [
    "http://bizlem.io/PurchaseOrderProcessing#CGST",
    20,
    1127,
    1771,
    1841
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#SGST",
    20,
    1127,
    2032,
    2102
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices",
    21,
    1141,
    347,
    504
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices",
    22,
    1152,
    354,
    506
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#Quantity",
    22,
    1152,
    1059,
    1099
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#Unit",
    22,
    1152,
    1135,
    1320
  ]
]


In [ ]:
print(json.dumps(slist[36], indent=2))

In [21]:
header_line_index = list(set([ line_index for (_, line_index, _, _, _) in main_table_headers ]))[:4]
header_scan_index = [ len(slist[line_index]['words']) - 1 for line_index in header_line_index ]
header_lines = []
header_boxes = []

while True:
    right_most_line = None
    right_x = 0
    for i, line_index in enumerate(header_line_index):
        if header_scan_index[i] < 0: continue
        words = slist[line_index]['words']
        if words[header_scan_index[i]]['x2'] > right_x:
            right_x = words[header_scan_index[i]]['x2']
            right_most_line = i
    
    print(header_scan_index, right_most_line)
    is_middle = True
    arr = [ ' ' ] * len(header_line_index)
    
    current_line_index = header_line_index[right_most_line]
    current_word_index = header_scan_index[right_most_line]
    current_word = slist[current_line_index]['words'][current_word_index]
    
    arr[right_most_line] = current_word['word']
    
    left_x = current_word['x1']
    right_x = current_word['x2']
    y = slist[current_line_index]['y']
    
    print('\t', current_word['word'], y, left_x, right_x)
    
    is_middle = True
    for i, line_index in enumerate(header_line_index):
        if i == right_most_line or header_scan_index[i] < 0: continue
        words = slist[line_index]['words']
        word_index = header_scan_index[i]
        
        if words[word_index]['x2'] > current_word['x1']:
            arr[i] = words[word_index]['word']
            header_scan_index[i] -= 1
            
            left_x = min(left_x, words[word_index]['x1'])
            right_x = max(right_x, words[word_index]['x2'])
            
            is_middle = False
    
    print('\tis_middle', is_middle)
    if is_middle:
        if current_word_index >= 1:
            prev_word = slist[current_line_index]['words'][current_word_index - 1]
            mid_word_x = (current_word['x2'] + prev_word['x1']) // 2
            
            for i, line_index in enumerate(header_line_index):
                if i >= right_most_line or header_scan_index[i] < 0: continue
                words = slist[line_index]['words']
                word_index = header_scan_index[i]
                
                if words[word_index]['x1'] <= mid_word_x and words[word_index]['x2'] >= mid_word_x:
                    arr[i] = words[word_index]['word']
                    left_x = min(left_x, words[word_index]['x1'])
                    right_x = max(right_x, words[word_index]['x2'])
    
    valid_arr = False
    for word in arr:
        if re.search('[a-zA-Z]+', word) is not None:
            valid_arr = True
            break
            
    if valid_arr:
        header_lines.append(arr)
        header_boxes.append((current_line_index, y, left_x, right_x))
    
    header_scan_index[right_most_line] -= 1
    
    print(arr, y, left_x, right_x)
    
    negative_header_scan_index = [ scan_index for scan_index in header_scan_index if scan_index < 0 ]
    if len(negative_header_scan_index) == len(header_scan_index):
        break
    
    print()

for header_arr in header_lines:
    query_param = dict([ ('H' + str(4 - i), (header_arr[-(i + 1)] if i < len(header_arr) else ' ')) for i in range(4) ])
    print('%30s   %s' % (header_arr, query_param))
    

[5, 2, 8] 2
	 Amt 1152 2321 2381
	is_middle True
[' ', ' ', 'Amt'] 1152 2321 2381

[5, 2, 7] 2
	 Total 1152 2232 2308
	is_middle True
[' ', ' ', 'Total'] 1152 2232 2308

[5, 2, 6] 0
	 SGST 1127 2032 2102
	is_middle True
['SGST', ' ', ' '] 1127 2032 2102

[4, 2, 6] 0
	 CGST 1127 1771 1841
	is_middle True
['CGST', ' ', ' '] 1127 1771 1841

[3, 2, 6] 0
	 Taxable 1127 1509 1610
	is_middle True
['Taxable', ' ', ' '] 1127 1509 1610

[2, 2, 6] 0
	 nical 1127 1382 1466
	is_middle False
['nical', ' ', 'Disc.'] 1127 1382 1466

[1, 2, 5] 2
	 Unit Rate 1152 1135 1320
	is_middle True
[' ', ' ', 'Unit Rate'] 1152 1135 1320

[1, 2, 4] 2
	 Qty 1152 1059 1099
	is_middle False
['PER', ' ', 'Qty'] 1152 985 1099

[0, 2, 3] 0
	 HSNI 1127 867 968
	is_middle False
['HSNI', ' ', 'can'] 1127 867 968

[-1, 2, 2] 2
	 Goods/Services 1152 558 784
	is_middle False
[' ', 'Goodata', 'Goods/Services'] 1152 558 784

[-1, 1, 1] 1
	 of 1141 523 551
	is_middle False
[' ', 'of', 'of'] 1141 523 551

[-1, 0, 0] 2
	 Descripti

In [17]:
print([ word['word'] for word in slist[18]['words'] ])
# print([ word['word'] for word in slist[25]['words'] ])

['GSTIN ID', ':', '24AAACE1741P1ZN', 'GSTIN', 'ID', ':24AAACE1741P1ZN']


In [27]:
def request_header(header_arr, header_index, res_header):
    try:
        query_param = dict([ ('H' + str(4 - i), (header_arr[-(i + 1)] if i < len(header_arr) else ' ')) for i in range(4) ])
        print(query_param)
        r = requests.post(FLATTEN_URL, data=json.dumps(query_param))
        
        r = r.json()
        if len(r['response']['docs']) > 0:
            phrase = r['response']['docs'][0]['url']
            res_header[header_index] = phrase
            return phrase
        else:
            res_header[header_index] = None
            return None
            
    except Exception as e:
        traceback.print_exc()
    
res_header = [ None ] * len(header_lines)
threads = [ threading.Thread(target=request_header, args=(header_arr, header_index, res_header)) for header_index, header_arr in enumerate(header_lines) ]
print(len(threads))

print('requesting...')
for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()
print('finish requesting...')
    
print(res_header)
print(header_boxes)

12
requesting...
{'H4': 'Amt', 'H3': ' ', 'H2': ' ', 'H1': ' '}
{'H4': 'Total', 'H3': ' ', 'H2': ' ', 'H1': ' '}
{'H4': ' ', 'H3': ' ', 'H2': 'SGST', 'H1': ' '}
{'H4': ' ', 'H3': ' ', 'H2': 'CGST', 'H1': ' '}
{'H4': ' ', 'H3': ' ', 'H2': 'Taxable', 'H1': ' '}
{'H4': 'Disc.', 'H3': ' ', 'H2': 'nical', 'H1': ' '}
{'H4': 'Unit Rate', 'H3': ' ', 'H2': ' ', 'H1': ' '}
{'H4': 'Qty', 'H3': ' ', 'H2': 'PER', 'H1': ' '}
{'H4': 'can', 'H3': ' ', 'H2': 'HSNI', 'H1': ' '}
{'H4': 'Goods/Services', 'H3': 'Goodata', 'H2': ' ', 'H1': ' '}
{'H4': 'of', 'H3': 'of', 'H2': ' ', 'H1': ' '}
{'H4': 'Description', 'H3': 'Description', 'H2': ' ', 'H1': ' '}
finish requesting...
[['http://bizlem.io/PurchaseOrderProcessing#AMOUNT'], ['http://bizlem.io/PurchaseOrderProcessing#TOTAL'], ['http://bizlem.io/PurchaseOrderProcessing#SGST'], ['http://bizlem.io/PurchaseOrderProcessing#CGST'], ['http://bizlem.io/PurchaseOrderProcessing#TaxableAmount'], ['nical  Disc.'], ['http://bizlem.io/PurchaseOrderProcessing#Unit'], N

Traceback (most recent call last):
  File "<ipython-input-27-fb76830d5a59>", line 7, in request_header
    r = r.json()
  File "/usr/lib/python3.7/site-packages/requests/models.py", line 889, in json
    self.content.decode(encoding), **kwargs
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [28]:
main_res_header = [ arr for arr in res_header if arr is not None ]
main_header_boxes = [ header_boxes[i] for i in range(len(res_header)) if res_header[i] is not None ]
res_header_url = [ None ] * len(main_res_header)

print(main_res_header)
print(main_header_boxes)
print()

def request_chosen_header(main_res_header, res_header_url):
    try:
        header_query = dict([ ('column' + str(len(main_res_header) - i), harr[0]) for i, harr in enumerate(main_res_header) ])
        print(json.dumps(header_query, indent=2))
        r = requests.post(CHOOSE_URL, data=json.dumps(header_query))
#         print('hello', r.text)
        print('json', json.dumps(r.json(), indent=2))
        r = r.json()['response']['docs'][0]['RequiredColumns'][0]
        r = json.loads(r)
        print(json.dumps(r, indent=2))
        
        for col_index, col_url in r.items():
            res_index = int(col_index[6:])
            res_header_url[-res_index] = col_url

        return r
            
    except Exception as e:
        traceback.print_exc()
    
request_chosen_header(main_res_header, res_header_url)
res_header_url

[['http://bizlem.io/PurchaseOrderProcessing#AMOUNT'], ['http://bizlem.io/PurchaseOrderProcessing#TOTAL'], ['http://bizlem.io/PurchaseOrderProcessing#SGST'], ['http://bizlem.io/PurchaseOrderProcessing#CGST'], ['http://bizlem.io/PurchaseOrderProcessing#TaxableAmount'], ['nical  Disc.'], ['http://bizlem.io/PurchaseOrderProcessing#Unit'], ['HSNI  can'], ['Goodata Goods/Services'], ['http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices'], ['http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices']]
[(22, 1152, 2321, 2381), (22, 1152, 2232, 2308), (20, 1127, 2032, 2102), (20, 1127, 1771, 1841), (20, 1127, 1509, 1610), (20, 1127, 1382, 1466), (22, 1152, 1135, 1320), (20, 1127, 867, 968), (22, 1152, 558, 784), (21, 1141, 523, 551), (22, 1152, 347, 506)]

{
  "column11": "http://bizlem.io/PurchaseOrderProcessing#AMOUNT",
  "column10": "http://bizlem.io/PurchaseOrderProcessing#TOTAL",
  "column9": "http://bizlem.io/PurchaseOrderProcessing#SGST",
  "column8": "http://bizlem.io/Pu

['http://bizlem.io/PurchaseOrderProcessing#AMOUNT',
 'http://bizlem.io/PurchaseOrderProcessing#TOTAL',
 'http://bizlem.io/PurchaseOrderProcessing#SGST',
 'http://bizlem.io/PurchaseOrderProcessing#CGST',
 'http://bizlem.io/PurchaseOrderProcessing#TaxableAmount',
 'nical  Disc.',
 'http://bizlem.io/PurchaseOrderProcessing#Unit',
 'HSNI  can',
 'Goodata Goods/Services',
 'http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices',
 'http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices']

In [29]:
for i, header in enumerate(res_header_url):
    print(i, header, main_header_boxes[i])
    
print(len(main_header_boxes), len(res_header_url), len(main_res_header))

0 http://bizlem.io/PurchaseOrderProcessing#AMOUNT (22, 1152, 2321, 2381)
1 http://bizlem.io/PurchaseOrderProcessing#TOTAL (22, 1152, 2232, 2308)
2 http://bizlem.io/PurchaseOrderProcessing#SGST (20, 1127, 2032, 2102)
3 http://bizlem.io/PurchaseOrderProcessing#CGST (20, 1127, 1771, 1841)
4 http://bizlem.io/PurchaseOrderProcessing#TaxableAmount (20, 1127, 1509, 1610)
5 nical  Disc. (20, 1127, 1382, 1466)
6 http://bizlem.io/PurchaseOrderProcessing#Unit (22, 1152, 1135, 1320)
7 HSNI  can (20, 1127, 867, 968)
8 Goodata Goods/Services (22, 1152, 558, 784)
9 http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices (21, 1141, 523, 551)
10 http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices (22, 1152, 347, 506)
11 11 11


In [28]:
header_pos = []
start_line_index = 0

MARGIN_HEADER = {
    'SRNO': 5,
    'HSN/SAC': 8,
    'Quantity': 6,
    'Rate': 15,
    'Unit': 5,
    'AMOUNT': 15,
    'PreTaxAmount': 15,
    'SGSTPercent': 4,
    'CGSTPercent': 4,
    'IGSTPercent': 4,
    'SGSTAmt': 15,
    'CGSTAmt': 15,
    'IGSTAmt': 15,
    'SGST': 15,
    'CGST': 15,
    'IGST': 15,
    'TOTAL': 10,  
}


for i, (line_index, y, left_x, right_x) in enumerate(main_header_boxes):
    if res_header_url[i] is None: continue
    header = res_header_url[i]
    header_pos.append((header, line_index, y, [left_x, right_x]))
    if start_line_index < line_index:
        start_line_index = line_index
    
header_pos.sort(key=lambda x: x[-1][0])
for header, _, _, x_pos in header_pos:
    key = header.split('#')[-1]

    if 'unknown' in header or key not in MARGIN_HEADER:
        x_pos[0] -= 8
        x_pos[1] += 8
    elif 'Description' not in header:
        x_pos[0] -= MARGIN_HEADER[key]
        x_pos[1] += MARGIN_HEADER[key]
        
header_pos[-1][-1][-1] = max_x

for i, (header, _, _, x_pos) in enumerate(header_pos):
    if 'Description' in header:
        if i > 0:
            x_pos_prev = header_pos[i - 1][-1]
            x_pos[0] = x_pos_prev[1] + 1
        if i < len(header_pos) - 1:
            x_pos_next = header_pos[i + 1][-1]
            x_pos[1] = x_pos_next[0] - 1

stop_line_index = start_line_index + 1

end = False
for line_index, words in enumerate(slist):
    if line_index <= stop_line_index or len(sres[line_index]['words']) == 0: 
        continue
        
    for word_index, word in enumerate(words['words']):
        url = sres_words[line_index]['words'][word_index]
        if SELECTED_LABEL not in url: continue
            
        url_key = url.split('#')[-1]
        if url_key == 'GrossTotal' or 'SGST' in url_key or 'CGST' in url_key or 'IGST' in url_key: 
            stop_line_index = line_index
            end = True
            break
    if end:
        break

# print(json.dumps(header_pos, indent=2))
print(start_line_index, stop_line_index)
print(header_pos)

39 66
[('http://bizlem.io/PurchaseOrderProcessing#SRNO', 38, 1523, [133, 175]), ('http://bizlem.io/PurchaseOrderProcessing#Description', 39, 1548, [176, 713]), ('http://bizlem.io/PurchaseOrderProcessing#HSN/SAC', 38, 1523, [714, 794]), ('http://bizlem.io/PurchaseOrderProcessing#Quantity', 39, 1548, [856, 916]), ('JUOM', 39, 1548, [948, 1038]), ('http://bizlem.io/PurchaseOrderProcessing#Rate', 39, 1548, [1080, 1175]), ('http://bizlem.io/PurchaseOrderProcessing#SGST', 39, 1548, [1887, 2002]), ('http://bizlem.io/PurchaseOrderProcessing#IGST', 39, 1548, [2046, 2377])]


In [29]:
items_url = []
items_all = []

def createItem():
    keys = [
        'SRNO',
        'Description',
        'HSN/SAC',
        'Quantity',
        'Rate',
        'Unit',
        'AMOUNT',
        'PreTaxAmount',
        'SGSTPercent',
        'CGSTPercent',
        'IGSTPercent',
        'SGSTAmt',
        'CGSTAmt',
        'IGSTAmt',
        'TOTAL',
        'SGST',
        'CGST',
        'IGST',
    ]
    
    item = {}
    for key in keys:
        item[SELECTED_LABEL + key] = ''
    return item

for words_line, words in enumerate(slist[start_line_index + 1:stop_line_index]):
    item_url = createItem()
    item_all = {}
    
    for word in words['words']:
        x1, x2 = word['x1'], word['x2']
        
        for header, _, _, pos in header_pos:
            if x1 >= pos[0] and x2 <= pos[1]:
                desc = word['word']
                
                item_all.setdefault(header, '')
                    
                if 'unknown' not in header and header in item_url:
                    item_url[header] += desc + ' '
                
                item_all[header] += desc + ' '
                
    line_no = words_line + start_line_index + 1
    item_url['line_no'] = line_no
    item_all['line_no'] = line_no
    items_url.append(item_url)
    items_all.append(item_all)
    
print(json.dumps(items_all, indent=2))
print(json.dumps(items_url, indent=2))

[
  {
    "http://bizlem.io/PurchaseOrderProcessing#IGST": "Value ",
    "line_no": 40
  },
  {
    "line_no": 41
  },
  {
    "http://bizlem.io/PurchaseOrderProcessing#IGST": "3,600.00 23,600.00 ",
    "line_no": 42
  },
  {
    "http://bizlem.io/PurchaseOrderProcessing#SRNO": "1 ",
    "http://bizlem.io/PurchaseOrderProcessing#Description": "\"Titan\"brand chain pulley block,triple ",
    "JUOM": "NOS ",
    "http://bizlem.io/PurchaseOrderProcessing#Rate": "4, ",
    "line_no": 43
  },
  {
    "http://bizlem.io/PurchaseOrderProcessing#Description": "spur geared, compact, Cap: 1 Ton x 3 ",
    "line_no": 44
  },
  {
    "http://bizlem.io/PurchaseOrderProcessing#Description": "Mtrs lift. ",
    "line_no": 45
  },
  {
    "http://bizlem.io/PurchaseOrderProcessing#Description": "chain pulley block, triple ",
    "JUOM": "NOS ",
    "http://bizlem.io/PurchaseOrderProcessing#IGST": "4,500.00 29,500.00 ",
    "line_no": 46
  },
  {
    "line_no": 47
  },
  {
    "http://bizlem.io/PurchaseOr

In [16]:
footer_headers = {
    'GrossTotal': number_regex,
    'TOTAL': number_regex,
    'SGSTAmt': number_regex,
    'CGSTAmt': number_regex,
    'IGSTAmt': number_regex,
    'TotalAmt': number_regex,
    'CGSTAmount': number_regex,
    'SGSTAmount': number_regex,
    'IGSTAmount': number_regex,
    'CGST': percentage_regex,
    'SGST': percentage_regex,
    'IGST': percentage_regex,
}

footer_lines = []
for lindex, line in enumerate(slist[stop_line_index:]):
    line_index = lindex + stop_line_index
    item = {}
    for word_index, words in enumerate(line['words']):
        url = sres_words[line_index]['words'][word_index]
        if SELECTED_LABEL in url:
            term = url.split('#')[-1]
            if term in footer_headers:
                paragraph = concat(slist, line_index, word_index, n_lines=0)
                regex = footer_headers[term]
                search_p = re.findall(number_regex, paragraph)
                if len(search_p) > 0:
                    item[url] = [ to_float(i) for i in search_p ]
    
    if len(item.keys()) == 0:
        continue
        
    item['line_no'] = line_index
    footer_lines.append(item)
    

    

In [17]:

table_output = {
    'header_table_only_url': items_url,
    'header_table_all': items_all,
    'footer_table_only_url': footer_lines,
}

print(json.dumps(table_output, indent=2))

{
  "header_table_only_url": [
    {
      "http://bizlem.io/PurchaseOrderProcessing#SRNO": "",
      "http://bizlem.io/PurchaseOrderProcessing#Description": "Services ",
      "http://bizlem.io/PurchaseOrderProcessing#HSN/SAC": "",
      "http://bizlem.io/PurchaseOrderProcessing#Quantity": "",
      "http://bizlem.io/PurchaseOrderProcessing#Rate": "",
      "http://bizlem.io/PurchaseOrderProcessing#Unit": "",
      "http://bizlem.io/PurchaseOrderProcessing#AMOUNT": "",
      "http://bizlem.io/PurchaseOrderProcessing#PreTaxAmount": "",
      "http://bizlem.io/PurchaseOrderProcessing#SGSTPercent": "",
      "http://bizlem.io/PurchaseOrderProcessing#CGSTPercent": "",
      "http://bizlem.io/PurchaseOrderProcessing#IGSTPercent": "",
      "http://bizlem.io/PurchaseOrderProcessing#SGSTAmt": "",
      "http://bizlem.io/PurchaseOrderProcessing#CGSTAmt": "",
      "http://bizlem.io/PurchaseOrderProcessing#IGSTAmt": "",
      "http://bizlem.io/PurchaseOrderProcessing#TOTAL": "",
      "http://

In [32]:
def request_validation(items):
#     try:
        query_params = {
            'header_table': items,
            'footer_table': []
        }
        
        r = requests.post(VALIDATION_URL, data=json.dumps(query_param))
        
        r = r.text
        print(r)
            
#     except Exception as e:
#         print(items, 'error request_validation')
#         print(e)
    
request_validation(items)

NameError: name 'items' is not defined

In [ ]:
slist

# Regex for Invoice Number 

In [22]:
def concat_rows_and_cols(slist, line_index, word_index, n_lines=10):
    paragraph = []
    for word in slist[line_index]['words'][word_index:]:
        nword = word.copy()
        nword['y'] = slist[line_index]['y']
        nword['line_index'] = line_index
        paragraph.append(nword)
    
    current_word = slist[line_index]['words'][word_index]
    
    for lindex in range(line_index + 1, min(line_index + n_lines + 1, len(slist))):
        line = slist[lindex]
        for word in line['words']:
            if word['x1'] >= current_word['x1'] - 15 and word['x1'] <= current_word['x2']:
                nword = word.copy()
                nword['y'] = line['y']
                nword['line_index'] = lindex
                paragraph.append(nword)
    
    return paragraph

def request_regex_word(word, word_index, container):
    try:
        r = requests.post(REGEX_URL, data=word['word'])
        container[word_index] = r.json()
    except Exception as e:
        traceback.print_exc()

def request_regex_header(slist, line_index, word_index, word_url, paragraph, regex_res):
    container = [ None ] * len(paragraph)
    threads = [ threading.Thread(target=request_regex_word, args=(word, word_index, container)) for word_index, word in enumerate(paragraph) ]
    for thread in threads:
        thread.start()
    
    for thread in threads:
        thread.join()
        
    print(container[1], paragraph[1])

    regex_res.setdefault(word_url, {
        'Regex': [],
        'Integer': [],
        'Date': [],
        'UKS': [],
    })
    
    current_word = slist[line_index]['words'][word_index]
    for rr_index, rr in enumerate(container):
        word = paragraph[rr_index]
        for item in rr:
            regex_res[word_url]['Regex'].append({
                'Regex_Type': item['Regex_Type'],
                'Regex_Value': item['value'],
                'Regex_Id': item['Regex_id'],
                'Regex_Length': item['length'],
                'Regex_Line_OffSet': word['line_index'] - line_index,
                'Regex_X1 OffSet': word['x1'] - current_word['x1'],
                'Regex_X1-X2 OffSet': word['x1'] - current_word['x2'],
                'Regex_Y OffSet': word['y'] - slist[line_index]['y'],
                'Regex_X2-X1 OffSet': word['x2'] - current_word['x1'],
                'Regex_X2-X2 OffSet': word['x2'] - current_word['x2'],
            })
            
    for word in paragraph:
        search_result = re.search('\d+', word['word'])
        if search_result is not None:
            regex_res[word_url]['Integer'].append({
                'Integer_Value': word['word'][search_result.start():search_result.end()],
                'Integer_Length': search_result.end() - search_result.start(),
                'Integer_Line_OffSet': word['line_index'] - line_index,
                'Integer_X1 OffSet': word['x1'] - current_word['x1'],
                'Integer_X1-X2 OffSet': word['x1'] - current_word['x2'],
                'Integer_Y OffSet': word['y'] - slist[line_index]['y'],
                'Integer_X2-X1 OffSet': word['x2'] - current_word['x1'],
                'Integer_X2-X2 OffSet': word['x2'] - current_word['x2'],
            })
            
        search_result_date = re.search(date_regex, word['word'])
        if search_result_date is not None:
            regex_res[word_url]['Date'].append({
                'Date_Value': word['word'][search_result_date.start():search_result_date.end()],
                'Date_Length': search_result_date.end() - search_result_date.start(),
                'Date_Line_OffSet': word['line_index'] - line_index,
                'Date_X1 OffSet': word['x1'] - current_word['x1'],
                'Date_X1-X2 OffSet': word['x1'] - current_word['x2'],
                'Date_Y OffSet': word['y'] - slist[line_index]['y'],
                'Date_X2-X1 OffSet': word['x2'] - current_word['x1'],
                'Date_X2-X2 OffSet': word['x2'] - current_word['x2'],
            })

# paragraph = concat_rows_and_cols(slist, 65, 1)
# print(paragraph)
# regex_res = {}
# request_regex_header(slist, 65, 1, sres_words[65]['words'][1], paragraph, regex_res)
# regex_res

In [24]:
regex_urls = [
    "http://bizlem.io/PurchaseOrderProcessing#Invoice_Number",
    "http://bizlem.io/PurchaseOrderProcessing#Invoice_Date",
    "http://bizlem.io/PurchaseOrderProcessing#PAN_Number",
    "http://bizlem.io/PurchaseOrderProcessing#Purchase_Order_Number",
    "http://bizlem.io/PurchaseOrderProcessing#TAN",
    "http://bizlem.io/PurchaseOrderProcessing#GSTIN",
    "http://bizlem.io/PurchaseOrderProcessing#TRNO",
    "http://bizlem.io/PurchaseOrderProcessing#TRDate",
    "http://bizlem.io/PurchaseOrderProcessing#CIN",
    "http://bizlem.io/PurchaseOrderProcessing#VAT",
    "http://bizlem.io/PurchaseOrderProcessing#PODate",
    "http://bizlem.io/PurchaseOrderProcessing#Date",
]



threads = []
regex_res = {}
for line_index, line in enumerate(slist):
#     print(line)
    for word_index, word in enumerate(line['words']):
#         print('\t', line_index, word_index)
        word_url = sres_words[line_index]['words'][word_index]
        if word_url in regex_urls:
            print(line_index, word_index, word_url)
            paragraph = concat_rows_and_cols(slist, line_index, word_index)
            
            threads.append(threading.Thread(
                target=request_regex_header, 
                args=(slist, line_index, word_index, word_url, paragraph, regex_res),
            ))
            
for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()
    
print(json.dumps(regex_res, indent=2))
            

1 1 http://bizlem.io/PurchaseOrderProcessing#Invoice_Number
4 0 http://bizlem.io/PurchaseOrderProcessing#PAN_Number
6 2 http://bizlem.io/PurchaseOrderProcessing#Invoice_Date
20 0 http://bizlem.io/PurchaseOrderProcessing#GSTIN
22 0 http://bizlem.io/PurchaseOrderProcessing#Date
22 6 http://bizlem.io/PurchaseOrderProcessing#Purchase_Order_Number
46 0 http://bizlem.io/PurchaseOrderProcessing#Date
[] {'word': ':', 'x1': 244, 'x2': 251, 'y': 186, 'line_index': 4}
[{'Regex_Type': 'DIGITS', 'Regex_id': 'dg_4', 'length': 4, 'value': '2019'}, {'Regex_Type': 'Document', 'Regex_id': 'DOC_43', 'length': 5, 'value': '01-02'}] {'word': '01-02-2019', 'x1': 2450, 'x2': 2571, 'y': 200, 'line_index': 6}[{'Regex_Type': 'DIGITS', 'Regex_id': 'dg_2', 'length': 2, 'value': '01'}, {'Regex_Type': 'Document', 'Regex_id': 'DOC_43', 'length': 5, 'value': '01-02'}] {'word': '01-02', 'x1': 2523, 'x2': 2645, 'y': 1387, 'line_index': 46}

[] {'word': '&', 'x1': 1975, 'x2': 1983, 'y': 522, 'line_index': 22}
[] {'word'